In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d, KDTree
from tqdm import tqdm
from random import random
from math import sqrt 

# Load image
image_path = "/Users/nshelton/Pictures/daruma.jpg"

image = cv2.imread(image_path)
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# image_value = cv2.resize(image_gray, (-1, -1), fx = 0.5, fy = 0.5)
image_value = image_gray
image_value = (255 - image_value).astype(np.float32)
image_value -= image_value.min()
image_value /= image_value.max()


filtered = image_value * 255
# filtered = cv2.blur(filtered, (7,7)) 
# filtered = np.abs(cv2.Laplacian(filtered * 50, cv2.CV_32F))


filtered -= filtered.min()
filtered /= filtered.max()

filtered = (255 * filtered).astype(np.uint8) 

all_strokes = np.ones(filtered.shape) * 255
all_paths = []
for i in range(1000):
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(filtered)

    print(max_loc)

    def placeStroke(input_raster, start):
        img = input_raster.copy()
        path = []
        target_color = img[start[1], start[0]]
        direction = np.pi
        pos = np.array(start)

        window = 2
        last_diff = [0,0]
        for i in range(100):
            min_val, max_val, min_loc, max_loc  = cv2.minMaxLoc(img[pos[1] - window:pos[1] + window +1, pos[0] - window:pos[0] + window +1])
           
            # if (max_val < target_color / 2): 
                # break
            if (max_val < 10): 
                break

            pos += np.array(max_loc) - window
            path.append(pos.copy())
            cv2.circle(img, pos, window-1, 0, -1)

        print(path)
        return path
    
    line_thickness = 2
    def rasterStroke(dims, path) :
        img = np.zeros(dims)
        for i in range(len(path)-1):
            a = (round(path[i][0]) , round(path[i][1]) )
            b = (round(path[i+1][0]) , round(path[i+1][1]) )
            cv2.line(img, a, b, 1, line_thickness, cv2.LINE_AA)
            
        return img

    def smooth_stroke(path):
        iterations = 10
        for j in range(iterations):
            for i in range(1, len(path)-1):
                a = path[i-1]
                b = path[i+1]
                c = (a+b)/2
                path[i] = path[i] * 0.5 + c * 0.5

        return path

    
    new_stroke = placeStroke(filtered, max_loc)
    new_Stroke = smooth_stroke(new_stroke)
    all_paths.append(new_stroke)
    stroke_raster = rasterStroke(image_value.shape, new_stroke)


    filtered = filtered - stroke_raster * 100
    all_strokes = np.clip(all_strokes.astype(np.int32) - stroke_raster.astype(np.int32) * 100, 0,255).astype(np.uint8)

    cv2.imwrite("current_stroke.png",  (all_strokes).astype(np.uint8))

    cv2.imwrite("filtered.png", filtered)

image_value = (255 * image_value).astype(np.uint8)


cv2.imwrite("threshold.png", image_value)





(184, 297)
[array([184, 297]), array([186, 299]), array([187, 297]), array([188, 295]), array([186, 293]), array([188, 293]), array([190, 294]), array([192, 292]), array([193, 291]), array([191, 293]), array([189, 291]), array([190, 289]), array([188, 288]), array([190, 287]), array([191, 286]), array([193, 285]), array([194, 287]), array([196, 288]), array([194, 290]), array([192, 289]), array([193, 288]), array([195, 289]), array([197, 287]), array([199, 286]), array([201, 285]), array([203, 284]), array([205, 283]), array([207, 282]), array([209, 281]), array([211, 281]), array([213, 280]), array([215, 280]), array([217, 279]), array([219, 279]), array([221, 279]), array([223, 279]), array([225, 279]), array([227, 280]), array([229, 280]), array([231, 281]), array([233, 281]), array([234, 282]), array([236, 283]), array([237, 281]), array([238, 283]), array([239, 285]), array([241, 285]), array([242, 286]), array([244, 286]), array([244, 284]), array([246, 282]), array([248, 284]), 

True

In [3]:

import cv2 
import numpy as np

upscale = 10
def plotPaths(paths, path_scale = 1):
    img = np.ones((297 * upscale, 420 * upscale, 3), np.uint8) * 255

    for path in paths :
        for i in range(len(path)-1):
            a = (round(path[i][0]* upscale * path_scale) , round(path[i][1]* upscale * path_scale) )
            b = (round(path[i+1][0]* upscale * path_scale) , round(path[i+1][1]* upscale * path_scale) )
            cv2.line(img, a, b, (0,0,0), 1, cv2.LINE_AA)

    cv2.imwrite("render.png", img)

path_list = [[p.tolist() for p in path] for path in all_paths ]


plotPaths(path_list, 1)



In [4]:
 
# import json
# import uuid

# projects_folder = "/Users/nshelton/Hephaestus/projects"

# def writePlot(paths):
#     dictionary = {
#         "created_time": "4/7/2024 4:42:27 PM",
#         "modified_time": "4/7/2024 6:27:36 PM",
#         "camera_position": [100, 100],
#         "zoom": 100.702,
#         "aspect": 0.563,
#         "dom_element": {},
#         "plot_models": [],
#     }

#     dictionary["plot_models"].append(
#         {
#             "position": {"x": 0, "y": 0},
#             "paths": paths,
#             "scale": 0.1,
#             "id": str(uuid.uuid4()),
#             "state": "none",
#         }
#     )

#     with open(projects_folder + "/daruma2.json", "w") as outfile:
        
#         json.dump(dictionary, outfile)


# writePlot(path_list)

